## 2021: Week 13 - Premier League Statistics

Before Simon joined The Data School in the UK, he was a professional sporting performance analyst. Simon has reached into his previous professional life to come up with a football (read soccer) based challenge for this week. 

Simon is channelling his inner fanalyst to use data to understand more about the game that he enjoys. 

This week we want to create a data set that allows us to analyse 'Open Play Goals' scored. We will rank the players overall and by their position. 

### Input

5 csv files, all with a similar structure. There are a lot of columns in these data sets.

![img](https://1.bp.blogspot.com/-LgrltPPfiQI/YGGYbH3V12I/AAAAAAAACIw/9RiC8fLny98MIw2GjjagIg1DpZBfPAp7gCLcBGAsYHQ/w640-h206/Screenshot%2B2021-03-29%2Bat%2B10.05.25.png)

### Requirement
Open play goal scoring prowess in the Premier League 2015-2020
1. Input all the files
2. Remove all goalkeepers from the data set
3. Remove all records where appearances = 0	
4. In this challenge we are interested in the goals scored from open play
    - Create a new “Open Play Goals” field (the goals scored from open play is the number of goals scored that weren’t penalties or freekicks)
    - Note some players will have scored free kicks or penalties with their left or right foot
    - Be careful how Prep handles null fields! (have a look at those penalty and free kick fields) 
    - Rename the original Goals scored field to Total Goals Scored
5. Calculate the totals for each of the key metrics across the whole time period for each player, (be careful not to lose their position)
6. Create an open play goals per appearance field across the whole time period
7. Rank the players for the amount of open play goals scored across the whole time period, we are only interested in the top 20 (including those that are tied for position) – Output 1
8. Rank the players for the amount of open play goals scored across the whole time period by position, we are only interested in the top 20 (including those that are tied for position) – Output 2
9. Output the data – in your solution on twitter / the forums, state the name of the player who was the only non-forward to make it into the overall top 20 for open play goals scored

### Output

- Overall Rank
![img](https://1.bp.blogspot.com/-xc4fVyBVWO0/YGMhwxNq8LI/AAAAAAAACJA/lEK27Th2KfclGSFXaPXihdLtWUOLzuTPQCLcBGAsYHQ/w640-h126/Screenshot%2B2021-03-30%2Bat%2B14.03.36.png)

- Rank by Position
![img](https://1.bp.blogspot.com/-K97KWwGSF6A/YGMiYn0oLoI/AAAAAAAACJI/Ey8XzxlsS8E6YFVa0H7YTFHY_SPpkpLEACLcBGAsYHQ/w640-h122/Screenshot%2B2021-03-30%2Bat%2B14.06.08.png)

Two files:
1. Overall Rank
- 22 Rows (23 including headers)
- 10 Fields:
    - Open Play Goals
    - Goals with Right Foot
    - Goals with Left Foot 
    - Position
    - Appearances
    - Rank
    - Total Goals
    - Open Play Goals / Game
    - Headed Goals
    - Name
2. Rank by Position
    - 65 Rows (66 including headers)
    - 10 Fields : as per the first output file

In [529]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [530]:
pl_15_16 = pd.read_csv("./data/pl_15-16.csv")
pl_16_17 = pd.read_csv("./data/pl_16-17.csv")
pl_17_18 = pd.read_csv("./data/pl_17-18.csv")
pl_18_19 = pd.read_csv("./data/pl_18-19.csv")
pl_19_20 = pd.read_csv("./data/pl_19-20.csv")

In [531]:
df = pd.concat([pl_15_16,
                pl_16_17,
                pl_17_18,
                pl_18_19,
                pl_19_20], axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4247 entries, 0 to 973
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Season                  4247 non-null   object 
 1   Name                    4247 non-null   object 
 2   Position                4247 non-null   object 
 3   Appearances             4247 non-null   int64  
 4   Clean sheets            1824 non-null   float64
 5   Goals conceded          1824 non-null   float64
 6   Tackles                 3768 non-null   float64
 7   Tackle success %        2883 non-null   object 
 8   Last man tackles        1345 non-null   float64
 9   Blocked shots           3768 non-null   float64
 10  Interceptions           3768 non-null   float64
 11  Clearances              3768 non-null   float64
 12  Headed Clearance        3768 non-null   float64
 13  Clearances off line     1345 non-null   float64
 14  Recoveries              2883 non-null   f

In [532]:
df["Name"] = df.Name.str.strip()

In [533]:
df.columns

Index(['Season', 'Name', 'Position', 'Appearances', 'Clean sheets',
       'Goals conceded', 'Tackles', 'Tackle success %', 'Last man tackles',
       'Blocked shots', 'Interceptions', 'Clearances', 'Headed Clearance',
       'Clearances off line', 'Recoveries', 'Duels won', 'Duels lost',
       'Successful 50/50s', 'Aerial battles won', 'Aerial battles lost',
       'Own goals', 'Errors leading to goal', 'Assists', 'Passes',
       'Passes per match', 'Big chances created', 'Crosses',
       'Cross accuracy %', 'Through balls', 'Accurate long balls',
       'Yellow cards', 'Red cards', 'Fouls', 'Offsides', 'Goals',
       'Headed goals', 'Goals with right foot', 'Goals with left foot',
       'Hit woodwork', 'Goals per match', 'Penalties scored',
       'Freekicks scored', 'Shots', 'Shots on target', 'Shooting accuracy %',
       'Big chances missed', 'Saves', 'Penalties saved', 'Punches',
       'High Claims', 'Catches', 'Sweeper clearances', 'Throw outs',
       'Goal Kicks'],
     

In [534]:
# Remove all goalkeepers from the data set
goalkeepers = df.loc[df["Position"] == "Goalkeeper"].index
df = df.drop(goalkeepers, axis=0)
df = df.reset_index(drop=True)

In [535]:
# Remove all records where appearances = 0
appearances = df.loc[df["Appearances"] == 0].index
df = df.drop(appearances, axis=0)
df = df.reset_index(drop=True)
df.shape

(1457, 54)

In [536]:
# In this challenge we are interested in the goals scored from open play
# Create a new “Open Play Goals” field (the goals scored from open play is the number of goals scored that weren’t penalties or freekicks)
# Note some players will have scored free kicks or penalties with their left or right foot
# Be careful how Prep handles null fields! (have a look at those penalty and free kick fields) 
# Rename the original Goals scored field to Total Goals Scored
df.columns

Index(['Season', 'Name', 'Position', 'Appearances', 'Clean sheets',
       'Goals conceded', 'Tackles', 'Tackle success %', 'Last man tackles',
       'Blocked shots', 'Interceptions', 'Clearances', 'Headed Clearance',
       'Clearances off line', 'Recoveries', 'Duels won', 'Duels lost',
       'Successful 50/50s', 'Aerial battles won', 'Aerial battles lost',
       'Own goals', 'Errors leading to goal', 'Assists', 'Passes',
       'Passes per match', 'Big chances created', 'Crosses',
       'Cross accuracy %', 'Through balls', 'Accurate long balls',
       'Yellow cards', 'Red cards', 'Fouls', 'Offsides', 'Goals',
       'Headed goals', 'Goals with right foot', 'Goals with left foot',
       'Hit woodwork', 'Goals per match', 'Penalties scored',
       'Freekicks scored', 'Shots', 'Shots on target', 'Shooting accuracy %',
       'Big chances missed', 'Saves', 'Penalties saved', 'Punches',
       'High Claims', 'Catches', 'Sweeper clearances', 'Throw outs',
       'Goal Kicks'],
     

In [537]:
df["Open Play Goals"] = df["Goals with left foot"] + df["Goals with right foot"] + df["Headed goals"] - df["Penalties scored"] - df["Freekicks scored"] 
df = df.rename(columns={"Goals": "Total Goals"})

In [538]:
# Calculate the totals for each of the key metrics across the whole time period for each player, 
# (be careful not to lose their position)
grouped = df.groupby(["Name", "Position"])[["Open Play Goals", "Goals with right foot", "Goals with left foot", 
                                            "Appearances", "Total Goals", "Headed goals"]].agg("sum")

In [539]:
# Create an open play goals per appearance field across the whole time period
grouped["Open Play Goals/Game"] = grouped["Open Play Goals"] / grouped["Appearances"]
grouped

,,Open Play Goals,Goals with right foot,Goals with left foot,Appearances,Total Goals,Headed goals,Open Play Goals/Game
Name,Position,,,,,,,
Aaron Connolly,Forward,3.0,3.0,0.0,24,3,0.0,0.125000
Aaron Cresswell,Defender,0.0,1.0,5.0,124,6,0.0,0.000000
Aaron Lennon,Midfielder,6.0,5.0,1.0,81,6,0.0,0.074074
Aaron Mooy,Midfielder,5.0,3.0,3.0,67,6,0.0,0.074627
Aaron Ramsey,Midfielder,12.0,11.0,1.0,55,12,0.0,0.218182
...,...,...,...,...,...,...,...,...
Zlatan Ibrahimovic,Forward,15.0,11.0,2.0,33,17,4.0,0.454545
Álvaro Morata,Forward,16.0,6.0,3.0,47,16,7.0,0.340426
Álvaro Negredo,Forward,7.0,0.0,8.0,36,9,1.0,0.194444


In [540]:
# Rank the players for the amount of open play goals scored across the whole time period, we are only interested in the top 20 (including those that are tied for position)
output_1 = grouped.sort_values(by="Open Play Goals", ascending=False).reset_index()
output_1["Rank"] = output_1["Open Play Goals"].rank(ascending=False, method="first")

In [541]:
output_1 = output_1.head(20)
output_1 = output_1.loc[:, ["Name", "Position", "Rank", "Total Goals", "Open Play Goals", "Goals with right foot",
                            "Goals with left foot", "Headed goals", "Appearances", "Open Play Goals/Game"]]
output_1[["Rank" ,"Total Goals", "Open Play Goals", "Goals with right foot", "Goals with left foot", "Headed goals"]] = output_1[["Rank" ,"Total Goals", "Open Play Goals", 
                                                                                                                                  "Goals with right foot", "Goals with left foot", "Headed goals"]].astype(int)
output_1

,Name,Position,Rank,Total Goals,Open Play Goals,Goals with right foot,Goals with left foot,Headed goals,Appearances,Open Play Goals/Game
0,Sadio Mané,Forward,1,74,74,44,20,10,164,0.451220
1,Jamie Vardy,Forward,2,85,67,52,22,11,142,0.471831
2,Mohamed Salah,Forward,3,73,66,11,58,4,108,0.611111
3,Sergio Agüero,Forward,4,65,53,46,7,12,86,0.616279
4,Harry Kane,Forward,5,55,47,32,15,7,75,0.626667
5,Eden Hazard,Forward,6,48,40,35,13,0,138,0.289855
6,Romelu Lukaku,Forward,7,41,39,9,23,9,71,0.549296
7,Son Heung-Min,Forward,8,39,39,20,16,3,126,0.309524
8,Chris Wood,Forward,9,34,33,12,8,14,94,0.351064
9,Roberto Firmino,Forward,10,34,33,19,8,7,106,0.311321


In [542]:
output_1.to_csv("./output/Week12_output_1.csv")

In [543]:
output_2 = grouped.reset_index()
output_2["Rank by Position"] = output_2.groupby(["Position"])["Open Play Goals"].rank(ascending=False, method="first")

In [544]:
output_2["Rank by Position"] = output_2["Rank by Position"].astype(int)
output_2.loc[output_2["Position"] == "Midfielder"].sort_values(by="Rank by Position", ascending=True).head(20)

,Name,Position,Open Play Goals,Goals with right foot,Goals with left foot,Appearances,Total Goals,Headed goals,Open Play Goals/Game,Rank by Position
430,Kevin De Bruyne,Midfielder,31.0,27.0,9.0,152,36,0.0,0.203947,1
277,Georginio Wijnaldum,Midfielder,24.0,16.0,1.0,179,25,8.0,0.134078,2
185,David Silva,Midfielder,15.0,2.0,15.0,80,17,0.0,0.187500,3
416,Juan Mata,Midfielder,14.0,3.0,15.0,113,18,0.0,0.123894,4
419,Junior Stanislas,Midfielder,14.0,17.0,2.0,99,20,1.0,0.141414,5
13,Adam Lallana,Midfielder,13.0,5.0,7.0,101,13,1.0,0.128713,6
234,Erik Lamela,Midfielder,13.0,1.0,11.0,103,13,1.0,0.126214,7
358,Jesse Lingard,Midfielder,13.0,12.0,1.0,107,14,1.0,0.121495,8
571,Nathan Redmond,Midfielder,13.0,10.0,2.0,106,14,1.0,0.122642,9
626,Philippe Coutinho,Midfielder,13.0,13.0,1.0,40,15,1.0,0.325000,10


In [545]:
output_2.to_csv("./output/Week12_output_2.csv")